### 此檔案切分資料集

In [ ]:
# test_index.txt review_index/userId/appID; 111619 rows × 3 columns
# train_index.txt review_index/userId/appID; 904120rows × 3 columns
# val_index.txt review_index/userId/appID; 100457rows × 3 columns
# train_index_all.txt(中間檔案，包含完整資料集移除test之後剩的90%) review_index/userId/appID; 1004577rows × 3 columns

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
from time import sleep
from tqdm import tqdm, trange
import re
from itertools import chain

#### 1) 讀出userID最大數字 
#### 2) 從userID隨機抽11萬2073筆存到一個list(test_userID)，不可重複
#### 3) 若userID被包含在該list(test_userID)中，則隨機抽取該userID的任1筆評論(review_index)，並將它加入到test_index串列中
#### 4) 其他沒被加入test_index的review_index，則加入train_index串列中。
#### 5) 最後test_index、train_index串列記得先都存回csv


In [2]:
df = pd.read_csv("./Data/ALL_ORIGIN_DATA_review.csv")
df

,Unnamed: 0,review_index,userId,appId,review
0,0,0,0,5,Now I understand why so many low marks. Two ob...
1,1,1,1,5,Its good but the adds to get more blocks wont ...
2,2,2,2,5,I love the game my lil sis showed me it and iv...
3,3,3,3,5,Good game. It deserves 5 stars because it isn'...
4,4,4,4,5,"Dear people, I lovvvvvvvve it but why do u hav..."
...,...,...,...,...,...
1116191,1116191,1116191,47487,9684,Useful features keep being removed. Now it was...
1116192,1116192,1116192,14297,9684,Algorithm makes it impossible to grow your acc...
1116193,1116193,1116193,38611,9684,"This app is a lot of fun..I am new to it, but ..."
1116194,1116194,1116194,6009,9684,It's been an awesome app to use! As someone wh...


In [ ]:
## 檢查 userId 出現次數才用到

# mylist = df['userId'].tolist()
# myset = set(mylist)  #myset是另外一個列表，裡面的內容是mylist裡面的無重複 項
# bad_userId=[]
# for item in myset:
#     lst_count = mylist.count(item)
#     if lst_count < 3:
#         print("the %d has found %d" %(item, lst_count))
#         bad_userId.append(item)

In [10]:
# 1) 讀出userID最大數字
print(df['userId'].max())

lst = [] #所有userID
for i in range(0,df['userId'].max()+1):
    lst.append(int(i))

135330


In [ ]:
# 2) 從userID隨機抽11萬1619筆存到一個list(test_userID)，不可重複
test_userID = []
import random
test_userID = random.sample(lst, k=111619)
test_userID

In [13]:
# 3) 若userID被包含在該list(test_userID)中，則隨機抽取該userID的任1筆評論(review_index)，並將它加入到test_index串列中

userId_array = np.array(df['userId'])
test_index=[]

for count,i in enumerate(tqdm(test_userID)):
    a = np.where(userId_array == i)
    sample_reviewId = np.random.choice(a[0], size=1)
    test_index.append([i,sample_reviewId[0]]) 

100%|█████████████████████████████████████████████████████████████████████████| 111619/111619 [02:27<00:00, 757.51it/s]


In [14]:
test_index = np.array(test_index)
print(test_index) # [userID='85676', review_index='144251']
print(test_index[0][1])

[[104298 556989]
 [ 24488 276457]
 [100603 827705]
 ...
 [115038 789357]
 [ 74020 112901]
 [ 52738 394800]]
556989


In [33]:
from pandas.core.frame import DataFrame

test_index_lst = []
test_userId_lst = []
test_appId_lst = []

for i in range (len(test_index)):
    test_index_lst.append(test_index[i][1])
    test_userId_lst.append(test_index[i][0])
    test_appId_lst.append(df['appId'][i])
    
test_index_dic={'review_index' : test_index_lst,"userId" : test_userId_lst,"appId" : test_appId_lst}
test_index_df = DataFrame(test_index_dic)
test_index_df   

,review_index,userId,appId
0,556989,104298,5
1,276457,24488,5
2,827705,100603,5
3,292180,56006,5
4,794402,109560,5
...,...,...,...
111614,762316,108249,1110
111615,3507,3332,1110
111616,789357,115038,1110
111617,112901,74020,1110


In [34]:
test_index_df.to_csv('./Data/test_index.txt',index=False, encoding='utf-8') 

### 以下先從完整dataset移除10%的test set 存到 train_index_all.txt

In [20]:
#建一個包含原訓練集review_index的lst
review_index_lst = []
for i in range(0,df['review_index'].max()+1):
    review_index_lst.append(int(i))
len(review_index_lst)

1116196

In [22]:
# 從 review_index_lst 移除 "test_index_lst"(已經在測試集裡的review_index)

train_index_lst = list(set(test_index_lst) ^ set(review_index_lst))
len(train_index_lst)

1004577

In [35]:
train_userId_lst = []
train_appId_lst = []
for i in tqdm(train_index_lst):
    train_userId_lst.append(df['userId'][i])
    train_appId_lst.append(df['appId'][i])
len(train_userId_lst)

100%|████████████████████████████████████████████████████████████████████| 1004577/1004577 [00:09<00:00, 101805.37it/s]


1004577

In [24]:
# 確認 test 和 train 交集為空
print (len(set(test_index_lst)&(set(train_index_lst))))

0


In [26]:
# 確認 train_user數量 與 完整資料集用戶數量 相同
print(len(set(df['userId'].tolist())))
print(len(set(train_userId_lst)))

135331
135331


In [36]:
train_index_dic = {'review_index' : train_index_lst,"userId" : train_userId_lst,"appId" :train_appId_lst }
train_index_df = DataFrame(train_index_dic)
train_index_df   

,review_index,userId,appId
0,0,0,5
1,1,1,5
2,2,2,5
3,3,3,5
4,4,4,5
...,...,...,...
1004572,1116190,128992,9684
1004573,1116191,47487,9684
1004574,1116192,14297,9684
1004575,1116193,38611,9684


In [37]:
train_index_df.to_csv('./Data/train_index_all.txt',index=False, encoding='utf-8') 
# train_index_all.txt 原dataset中的90%

###  val: 從原本的train_index_all.txt 抽出10%當val


In [38]:
df_train = pd.read_csv("./Data/train_index_all.txt")
df_train

,review_index,userId,appId
0,0,0,5
1,1,1,5
2,2,2,5
3,3,3,5
4,4,4,5
...,...,...,...
1004572,1116190,128992,9684
1004573,1116191,47487,9684
1004574,1116192,14297,9684
1004575,1116193,38611,9684


In [39]:
# 共有 1008665 筆 train資料，透過index來抽，再對應到review_index
lst = [] 
for i in range(0,len(train_index_lst)):
    lst.append(i)

# 抽100865筆存到一個list(val_ID)，不可重複
val_ID = []
import random
val_ID = random.sample(lst, k=100457)
val_ID

[720176,
 783275,
 306216,
 66778,
 494222,
 699949,
 803212,
 559570,
 560034,
 788219,
 478704,
 397655,
 268943,
 371122,
 481590,
 935140,
 972264,
 454999,
 567861,
 830274,
 751385,
 780508,
 633740,
 845138,
 967878,
 128869,
 631658,
 713568,
 688600,
 898377,
 456325,
 220532,
 183518,
 689861,
 572168,
 916398,
 994350,
 702228,
 177611,
 766303,
 406970,
 899793,
 470424,
 59029,
 302229,
 251365,
 258937,
 244578,
 728613,
 745012,
 441716,
 244284,
 339024,
 529136,
 799865,
 638844,
 630694,
 45667,
 894323,
 683917,
 635411,
 286179,
 879806,
 877923,
 776850,
 837929,
 93619,
 545753,
 61876,
 907726,
 84763,
 769305,
 292058,
 94091,
 287535,
 270298,
 477174,
 838611,
 341768,
 791201,
 8873,
 966223,
 19032,
 717250,
 512975,
 691383,
 456521,
 986915,
 963780,
 513455,
 453624,
 130463,
 707561,
 232420,
 951176,
 443132,
 140625,
 857638,
 474083,
 5036,
 760064,
 468661,
 432034,
 991965,
 280083,
 377438,
 221011,
 137394,
 57899,
 46994,
 113992,
 985455,
 25416

In [40]:
# 把被抽到的 val_ID 對應的review_index/userId/appId，加入val_review_index串列中
val_review_index,val_userId,val_appId=[],[],[]
for count,i in enumerate(tqdm(val_ID)):
    val_review_index.append(df_train['review_index'][i])
    val_userId.append(df_train['userId'][i])
    val_appId.append(df_train['appId'][i])

100%|███████████████████████████████████████████████████████████████████████| 100457/100457 [00:01<00:00, 66309.11it/s]


In [41]:
from pandas.core.frame import DataFrame
val_index_dic={'review_index' : val_review_index,"userId" : val_userId, "appId" : val_appId}
val_index_df = DataFrame(val_index_dic)
val_index_df  

,review_index,userId,appId
0,800602,30304,6378
1,870651,103296,6998
2,340076,26498,3216
3,74159,53877,701
4,549663,29013,3920
...,...,...,...
100452,700298,127337,5377
100453,219702,56862,2148
100454,643323,37277,4791
100455,621739,48691,4718


In [42]:
val_index_df.to_csv('./Data/val_index.txt',index=False, encoding='utf-8') 

### 排除 val set 之後的 train set >> train_index.txt

In [45]:
new_train_re_index,new_train_userId,new_train_appId = [],[],[]

for count,i in enumerate(tqdm(new_index)):
    new_train_re_index.append(df_train['review_index'][i])
    new_train_userId.append(df_train['userId'][i])
    new_train_appId.append(df_train['appId'][i])

100%|███████████████████████████████████████████████████████████████████████| 904120/904120 [00:13<00:00, 67749.64it/s]


In [46]:
new_index = list(set(val_ID)^set(lst))
len(new_index)

904120

In [47]:
new_train_dic={'review_index' : new_train_re_index,"userId" : new_train_userId, "appId" : new_train_appId}
new_train_df = DataFrame(new_train_dic)
new_train_df  

,review_index,userId,appId
0,0,0,5
1,1,1,5
2,2,2,5
3,3,3,5
4,4,4,5
...,...,...,...
904115,1116189,31293,9684
904116,1116190,128992,9684
904117,1116191,47487,9684
904118,1116192,14297,9684


In [48]:
new_train_df.to_csv('./Data/train_index.txt',index=False, encoding='utf-8') 